<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Libraries/-Datasets" data-toc-modified-id="Importing-Libraries/-Datasets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Libraries/ Datasets</a></span><ul class="toc-item"><li><span><a href="#Importing-Libraries" data-toc-modified-id="Importing-Libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Importing Libraries</a></span></li><li><span><a href="#Importing-the-Dataset" data-toc-modified-id="Importing-the-Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Importing the Dataset</a></span></li></ul></li><li><span><a href="#Preparing-The-Dataset" data-toc-modified-id="Preparing-The-Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preparing The Dataset</a></span><ul class="toc-item"><li><span><a href="#Droping-The-Unwanted-Cols" data-toc-modified-id="Droping-The-Unwanted-Cols-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Droping The Unwanted Cols</a></span></li><li><span><a href="#Taking-Care-of-Sentiment-Column" data-toc-modified-id="Taking-Care-of-Sentiment-Column-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Taking Care of Sentiment Column</a></span></li><li><span><a href="#Cleaning-the-Text" data-toc-modified-id="Cleaning-the-Text-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Cleaning the Text</a></span></li><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Tokenization</a></span></li><li><span><a href="#Padding" data-toc-modified-id="Padding-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Padding</a></span></li><li><span><a href="#Train---Test-Split" data-toc-modified-id="Train---Test-Split-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Train - Test Split</a></span></li></ul></li><li><span><a href="#Model-Building" data-toc-modified-id="Model-Building-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model Building</a></span></li><li><span><a href="#Creating-Checkpoint" data-toc-modified-id="Creating-Checkpoint-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creating Checkpoint</a></span></li><li><span><a href="#Training-The-Model" data-toc-modified-id="Training-The-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Training The Model</a></span></li><li><span><a href="#Evaluating-The-Model" data-toc-modified-id="Evaluating-The-Model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluating The Model</a></span></li></ul></div>

# Importing Libraries/ Datasets


## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
from bs4 import BeautifulSoup
import math
from tensorflow.keras import layers
import tensorflow_datasets as tfds

## Importing the Dataset

In [2]:
cols = ["sentiment", "id", "date", "query", "user", "text"]

df = pd.read_csv('training.1600000.csv',names=cols, header=None,engine="python",encoding="latin1")

# Preparing The Dataset

## Droping The Unwanted Cols

In [3]:
df.drop(['id', 'date', 'query', 'user'], axis = 1, inplace = True)

In [4]:
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df.shape

(1600000, 2)

## Taking Care of Sentiment Column 

In [6]:
print(df['sentiment'].unique())

[0 4]


In [7]:
df_labels = df['sentiment'].values
df_labels[df_labels == 4] = 1
set(df_labels)

{0, 1}

## Cleaning the Text

In [8]:
import nltk
corpus = []
for i in range(0, 1600000):
    text = re.sub(r'@[A-Za-z0-9]+', ' ', df['text'][i])       #Removing @
    text = re.sub(r'https?://[A-Za-z0-9./]+', ' ', df['text'][i])     #Removing website(url) links

    text = re.sub('[^a-zA-Z!.?]', ' ', df['text'][i])
    text = text.lower()
    text = text.split()    
    text = ' '.join(text)
    corpus.append(text)

In [10]:
corpus

['switchfoot http twitpic.com y zl awww that s a bummer. you shoulda got david carr of third day to do it. d',
 'is upset that he can t update his facebook by texting it... and might cry as a result school today also. blah!',
 'kenichan i dived many times for the ball. managed to save the rest go out of bounds',
 'my whole body feels itchy and like its on fire',
 'nationwideclass no it s not behaving at all. i m mad. why am i here? because i can t see you all over there.',
 'kwesidei not the whole crew',
 'need a hug',
 'loltrish hey long time no see! yes.. rains a bit only a bit lol i m fine thanks how s you ?',
 'tatiana k nope they didn t have it',
 'twittera que me muera ?',
 'spring break in plain city... it s snowing',
 'i just re pierced my ears',
 'caregiving i couldn t bear to watch it. and i thought the ua loss was embarrassing . . . . .',
 'octolinz it it counts idk why i did either. you never talk to me anymore',
 'smarrison i would ve been the first but i didn t have a gun

In [44]:
!pip install Gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23


In [46]:
"""from gensim import corpora
corpora.MmCorpus.serialize("./Saving model trail 3", corpus)"""

'from gensim import corpora\ncorpora.MmCorpus.serialize("./Saving model trail 3", corpus)'

In [47]:
with open('corpuss.pickle', 'wb') as f:
    pickle.dump(corpus, f)

## Tokenization

In [12]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus, target_vocab_size=2**16)

data_inputs = [tokenizer.encode(sentence) for sentence in corpus]

## Padding

In [13]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,value=0,padding="post",maxlen=MAX_LEN)




In [14]:
data_inputs.shape

(1600000, 73)

## Train - Test Split

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_inputs, df_labels, test_size = 0.20, random_state = 100)

# Model Building

In [16]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D() # no training variable so we can
                                             # use the same layer for each
                                             # pooling step
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [17]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 300
NB_CLASSES = 2#len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 70
NB_EPOCHS = 1

In [18]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [19]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

# Creating Checkpoint

In [21]:
import os
checkpoint_path = "./Saving model trail 3"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback= tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_weights_only=True,verbose = 1)



# Training The Model

In [22]:
Dcnn.fit(X_train,y_train,batch_size=BATCH_SIZE,epochs=NB_EPOCHS,callbacks= [cp_callback])

18286/18286 [==============================] - ETA: 0s - loss: 0.4028 - accuracy: 0.8164
Epoch 00001: saving model to .\Saving model trail 3
18286/18286 [==============================] - 5177s 283ms/step - loss: 0.4028 - accuracy: 0.8164


# Evaluating The Model

In [23]:
results = Dcnn.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(results)

4572/4572 [==============================] - 110s 24ms/step - loss: 0.3776 - accuracy: 0.8312
[0.3775647282600403, 0.8311906456947327]


In [24]:
if (Dcnn(np.array([tokenizer.encode("get the hell out of here")]), training=False).numpy()>0.5) == False:
    
    print ("BAD MOOD")
else:
    print("GOOD MOOD")

BAD MOOD


In [25]:
tokenizer.encode("love")

[662]

In [28]:
!dir {checkpoint_dir}

 Volume in drive C has no label.
 Volume Serial Number is B2D3-B6E1

 Directory of C:\Users\user\Documents\Sentiment analysis

16-07-2021  19:44    <DIR>          .
16-07-2021  19:44    <DIR>          ..
16-07-2021  16:00    <DIR>          .ipynb_checkpoints
16-07-2021  19:39                97 checkpoint
16-07-2021  16:52    <DIR>          my_model.h5py
16-07-2021  13:37    <DIR>          saved_model_Dcnn
16-07-2021  16:42    <DIR>          Saving model trail 3
16-07-2021  19:39       161,732,102 Saving model trail 3.data-00000-of-00001
16-07-2021  19:39             2,726 Saving model trail 3.index
16-07-2021  19:44           109,418 Sentiment Analysis.ipynb
21-09-2019  02:36       238,803,811 training.1600000.csv
               5 File(s)    400,648,154 bytes
               6 Dir(s)  331,516,022,784 bytes free


In [41]:
"""
Reffer to checkpoint call back usage from : https://www.tensorflow.org/tutorials/keras/save_and_load

"""
model= DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)
model.load_weights(checkpoint_path)

In [40]:
X_test.shape

(320000, 73)

In [42]:
if (model(np.array([tokenizer.encode("get the hell out of here")]), training=False).numpy()>0.5) == False:
    
    print ("BAD MOOD")
else:
    print("GOOD MOOD")

BAD MOOD
